In [1]:
import pandas as pd
from malid import config, helpers, logger
from malid.datamodels import GeneLocus

In [2]:
df = pd.read_csv(config.paths.base_data_dir / "CoV-AbDab_260722.filtered.tsv", sep="\t")
df["rownum"] = range(df.shape[0])
df

,CDRH3,j_gene,v_gene,VHorVHH,Binds to,Doesn't Bind to,Neutralising Vs,Not Neutralising Vs,Protein + Epitope,Origin,cdr3_seq_aa_q_trim,cdr3_aa_sequence_trim_len,rownum
0,ARERGYSGYGAAYYFDY,IGHJ4,IGHV1-69,EVQLLESGTEVKKPGSSVKVSCKASGGTFSSYAISWVRQAPGQGLE...,SARS-CoV2_WT;SARS-CoV2_Omicron-BA1,NaN,SARS-CoV2_WT;SARS-CoV2_Omicron-BA1,NaN,S; RBD,"Phage Display (Ab, human, immune - SARS-CoV2)",ARERGYSGYGAAYYFDY,17,0
1,ASWLYGDPISFDY,IGHJ4,IGHV4-39,EVQLLESGPGLVKPSETLSLTCTVSGGSISNSSYYWGWIRQPPGKG...,SARS-CoV2_WT;SARS-CoV2_Omicron-BA1,NaN,SARS-CoV2_WT;SARS-CoV2_Omicron-BA1,NaN,S; RBD,"Phage Display (Ab, human, immune - SARS-CoV2)",ASWLYGDPISFDY,13,1
2,AKSQGDYGGNSGAGYFQH,IGHJ1,IGHV3-9,EVQLVESGGGLVQPGRSLRLSCAASGFTFDEYAMHWVRQAPGKGLE...,SARS-CoV2_WT (weak);SARS-CoV2_Beta (weak);SARS...,NaN,SARS-CoV2_WT;SARS-CoV2_Gamma;SARS-CoV2_Omicron...,NaN,S; RBD,B-cells; SARS-CoV2_Gamma Human Patient,AKSQGDYGGNSGAGYFQH,18,2
3,ARLREFGDYYYYGMDV,IGHJ6,IGHV5-51,EVQLVQSGAEVKKPGESLKISCLGSGYSFTSYWIGWVRQMPGKGLE...,SARS-CoV2_WT (weak);SARS-CoV2_Beta,SARS-CoV2_WT,SARS-CoV2_Gamma;SARS-CoV2_Omicron-BA1 (weak),SARS-CoV2_WT,S; RBD,B-cells; SARS-CoV2_Gamma Human Patient,ARLREFGDYYYYGMDV,16,3
4,ARDTTDDYYVI,IGHJ1,IGHV4-61,QVQLQESGPGLVKPSQTLSLTCTVSGGSISSGSYYWSWIRQPAGKG...,SARS-CoV2_WT;SARS-CoV2_Beta;SARS-CoV2_Gamma,NaN,SARS-CoV2_WT;SARS-CoV2_Gamma,SARS-CoV2_Omicron-BA1,S; RBD,B-cells; SARS-CoV2_Gamma Human Patient,ARDTTDDYYVI,11,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6839,ARGESGSPYGMDV,IGHJ6,IGHV3-53,EVQLLESGGGLIQPGGSLRLSCAASGLTVSSNYMSWVRQAPGKGLE...,SARS-CoV2_WT,NaN,SARS-CoV2_WT,NaN,S; RBD,"Phage Display (Ab, human, immune - CoV2_WT)",ARGESGSPYGMDV,13,6839
6840,AAPSCSRTICSDGFDI,IGHJ3,IGHV1-58,EVQLVQSGPEVKKPGTSVKVACKASGFTFITPVSMQWVRQARGQRL...,SARS_CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS_CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,S; RBD,B-cells; SARS-CoV2 Human Vaccinees (BNT162b2),AAPSCSRTICSDGFDI,16,6840
6841,AREGRRYGSGWYISTGYFDY,IGHJ4,IGHV1-69,QVQLVQSGAEVKKPGSSVRVSCKASGGTFSTYPISWVRQAPGQGLE...,SARS-CoV2_WT,NaN,SARS-CoV2_WT,NaN,S; RBD,B-cells; SARS-CoV2_WT Human Patient,AREGRRYGSGWYISTGYFDY,20,6841
6842,AIHGGTYYYDKNILA,IGHJ4,IGHV3-30,EVQLVESGGGVVQPGTSLRLSCAASGFSFSHYVMYWVRQAPGKGLD...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,SARS-CoV2_Omicron,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,SARS-CoV2_Omicron,S; RBD,B-cells (SARS-CoV2 Vaccinated Human),AIHGGTYYYDKNILA,15,6842


In [3]:
df_parse = pd.concat(
    [
        pd.read_csv(fname, sep="\t")
        for fname in (config.paths.base_data_dir / "covabdab_igblast_splits").glob(
            "*.parsed.tsv"
        )
    ],
    axis=0,
)
# extract fasta ID
df_parse[["specimen_label", "rownum"]] = df_parse["query_id"].str.split(
    "|", expand=True
)
df_parse["rownum"] = df_parse["rownum"].astype(int)
for specimen_label, grp in df_parse.groupby("specimen_label"):
    assert not grp["rownum"].duplicated().any()
df_parse

,type,query_id,v_gene,FR1-IMGT,CDR1-IMGT,FR2-IMGT,CDR2-IMGT,FR3-IMGT,CDR3-IMGT (germline),specimen_label,rownum
0,V,covabdab|5000,IGHV3-30*16,QVQLVESGGGVVQPGRSLRLSCAAS,GFTFSTYA,MHWVRQAPGKGLEWVAV,ISYDGSNK,YYADSVKGRFTISRDNPKNTLYLQMNSLRAEDTAVYYC,AR,covabdab,5000
1,V,covabdab|5001,IGHV3-30*16,QVQLVESGGGVVQPGRSLRLSCAAS,GFTFSTYA,MHWVRQAPGKGLEWVTL,ISYDGGNK,YYADSVKGRFTISRDNSKNTLYLQMNSLRAEDAAVYFC,AR,covabdab,5001
2,V,covabdab|5002,IGHV3-30*18,QVHLVESGGGVVQPGRSLRLSCAAS,GFTFSNYG,MHWVRQAPGKGLEWVAV,ISGDENNK,FYANSVKGRFTISRDNSKNTLSLQMNSLRPEDTARYYC,AK,covabdab,5002
3,V,covabdab|5003,IGHV1-24*01,QVQLVQSGAEVKKPGASVKVSCKVS,GYTLTELS,IHWVRQAPGKGLEWMGG,FDPEDAET,IYAQNFQGRVTMTEDTSTDTAYMELSSLRSEDTALYYC,A,covabdab,5003
4,V,covabdab|5004,IGHV4-4*08,QVQLEESGPGLVKPSETLSLTCTVS,GGSISSHY,WSWIRQPPGKGLEWIGY,IQDSGST,NYNPSLKSRVTISVDTSKNQFSLRLSSVTTADTAVYYC,VR,covabdab,5004
...,...,...,...,...,...,...,...,...,...,...,...
4995,V,covabdab|4995,IGHV1-2*02,QVQLVQSGAEVKNPGASVKVSCKAS,GYIFTNYY,IHWVRQAPGQGLEWVGW,IHSLSGGT,SYAQKFQGRVTLTRDASIRTAYMELSRLGSDDTALYYC,AR,covabdab,4995
4996,V,covabdab|4996,IGHV1-2*02,QVQLVQSGAEVKKPGASVKVSCKAS,GYIFTNYY,IHWVRQAPGQGLEWVGW,IHSLSGGT,SYAQKFQGRVTLTRDAPIRTAYMELSGLGSDDTAVYYC,AR,covabdab,4996
4997,V,covabdab|4997,IGHV3-7*03,EVQLVESGGGLVQPGGSLRLSCAAS,GFTFSFYW,MSWVRQAPGKGLEWVAN,IKQDGSEK,YYVDSVKGRSTISRDNAKNSLYLQMNSLRAEDTAVYYC,AR,covabdab,4997
4998,V,covabdab|4998,IGHV3-30*16,QVQLVESGGGVVQPGRSLRLSCAAS,GFTFSYYA,ILWFRQAPGKGLEWVAI,ISYDGSNK,YYADSVKGRFTISRDNSKNTLYLQMNSLRPEDTAVYYC,AR,covabdab,4998


In [4]:
df["rownum"].describe()

count    6844.000000
mean     3421.500000
std      1975.836953
min         0.000000
25%      1710.750000
50%      3421.500000
75%      5132.250000
max      6843.000000
Name: rownum, dtype: float64

In [5]:
df_parse["rownum"].describe()

count    6844.000000
mean     3421.500000
std      1975.836953
min         0.000000
25%      1710.750000
50%      3421.500000
75%      5132.250000
max      6843.000000
Name: rownum, dtype: float64

In [6]:
# Merge our IgBlast output
# Our IgBlast gives some different V gene calls, and provides CDR1+2 sequences

In [7]:
orig_shape = df.shape
df = pd.merge(
    df.drop(columns=["v_gene"]),
    df_parse.rename(
        columns={"CDR1-IMGT": "cdr1_seq_aa_q_trim", "CDR2-IMGT": "cdr2_seq_aa_q_trim"}
    )[["v_gene", "rownum", "cdr1_seq_aa_q_trim", "cdr2_seq_aa_q_trim"]],
    left_on=["rownum"],
    right_on=["rownum"],
    how="inner",
    validate="1:1",
)
assert df.shape[0] == min(orig_shape[0], df_parse.shape[0])

In [8]:
df.shape[0], orig_shape[0], df_parse.shape[0]

(6844, 6844, 6844)

In [9]:
df.head()

,CDRH3,j_gene,VHorVHH,Binds to,Doesn't Bind to,Neutralising Vs,Not Neutralising Vs,Protein + Epitope,Origin,cdr3_seq_aa_q_trim,cdr3_aa_sequence_trim_len,rownum,v_gene,cdr1_seq_aa_q_trim,cdr2_seq_aa_q_trim
0,ARERGYSGYGAAYYFDY,IGHJ4,EVQLLESGTEVKKPGSSVKVSCKASGGTFSSYAISWVRQAPGQGLE...,SARS-CoV2_WT;SARS-CoV2_Omicron-BA1,NaN,SARS-CoV2_WT;SARS-CoV2_Omicron-BA1,NaN,S; RBD,"Phage Display (Ab, human, immune - SARS-CoV2)",ARERGYSGYGAAYYFDY,17,0,IGHV1-69*10,GGTFSSYA,IIPILGIA
1,ASWLYGDPISFDY,IGHJ4,EVQLLESGPGLVKPSETLSLTCTVSGGSISNSSYYWGWIRQPPGKG...,SARS-CoV2_WT;SARS-CoV2_Omicron-BA1,NaN,SARS-CoV2_WT;SARS-CoV2_Omicron-BA1,NaN,S; RBD,"Phage Display (Ab, human, immune - SARS-CoV2)",ASWLYGDPISFDY,13,1,IGHV4-39*01,GGSISNSSYY,IYYSGST
2,AKSQGDYGGNSGAGYFQH,IGHJ1,EVQLVESGGGLVQPGRSLRLSCAASGFTFDEYAMHWVRQAPGKGLE...,SARS-CoV2_WT (weak);SARS-CoV2_Beta (weak);SARS...,NaN,SARS-CoV2_WT;SARS-CoV2_Gamma;SARS-CoV2_Omicron...,NaN,S; RBD,B-cells; SARS-CoV2_Gamma Human Patient,AKSQGDYGGNSGAGYFQH,18,2,IGHV3-9*01,GFTFDEYA,SSWNSGTI
3,ARLREFGDYYYYGMDV,IGHJ6,EVQLVQSGAEVKKPGESLKISCLGSGYSFTSYWIGWVRQMPGKGLE...,SARS-CoV2_WT (weak);SARS-CoV2_Beta,SARS-CoV2_WT,SARS-CoV2_Gamma;SARS-CoV2_Omicron-BA1 (weak),SARS-CoV2_WT,S; RBD,B-cells; SARS-CoV2_Gamma Human Patient,ARLREFGDYYYYGMDV,16,3,IGHV5-51*03,GYSFTSYW,IFPADSDT
4,ARDTTDDYYVI,IGHJ1,QVQLQESGPGLVKPSQTLSLTCTVSGGSISSGSYYWSWIRQPAGKG...,SARS-CoV2_WT;SARS-CoV2_Beta;SARS-CoV2_Gamma,NaN,SARS-CoV2_WT;SARS-CoV2_Gamma,SARS-CoV2_Omicron-BA1,S; RBD,B-cells; SARS-CoV2_Gamma Human Patient,ARDTTDDYYVI,11,4,IGHV4-61*02,GGSISSGSYY,IYTSGST


In [10]:
# sanity check
assert df["v_gene"].str.startswith("IGHV").all()
assert (df["cdr1_seq_aa_q_trim"].str.strip() == df["cdr1_seq_aa_q_trim"]).all()
assert (df["cdr2_seq_aa_q_trim"].str.strip() == df["cdr2_seq_aa_q_trim"]).all()

In [11]:
df["v_gene"].value_counts()

IGHV3-30*16      517
IGHV3-30*18      357
IGHV1-69*13      327
IGHV3-53*01      278
IGHV4-39*01      255
                ... 
IGHV3-15*04        1
IGHV4-30-4*07      1
IGHV4-59*07        1
IGHV3-9*02         1
IGHV3-9*03         1
Name: v_gene, Length: 128, dtype: int64

In [12]:
df["v_gene"] = df["v_gene"].str.split("*").str[0].astype("category")

In [13]:
df["v_gene"].value_counts()

IGHV3-30         1077
IGHV1-69          698
IGHV3-53          356
IGHV3-23          326
IGHV3-33          318
IGHV4-39          270
IGHV1-46          256
IGHV3-9           255
IGHV5-51          231
IGHV1-18          206
IGHV3-66          186
IGHV1-2           176
IGHV4-4           173
IGHV4-59          170
IGHV3-21          148
IGHV3-13          143
IGHV1-24          141
IGHV4-31          137
IGHV3-7           133
IGHV2-5           127
IGHV3-48          121
IGHV4-34          117
IGHV7-4-1          94
IGHV3-15           87
IGHV1-58           87
IGHV3-30-3         81
IGHV4-61           77
IGHV2-70           74
IGHV3-11           66
IGHV1-8            58
IGHV4-30-4         54
IGHV1-3            51
IGHV5-a            48
IGHV3-49           46
IGHV3-64D          45
IGHV3-20           30
IGHV3-43           26
IGHV3-74           25
IGHV4-b            21
IGHV2-26           20
IGHV4-30-2         16
IGHV3-64           15
IGHV6-1            15
IGHV2-70D          10
IGHV1-f             8
IGHV3-43D 

In [14]:
# Note: we don't know mutation rate and we don't know isotype
# TODO: improve IgBlast parser to get mutation rate
# TODO: can we subset to matches in our dataset, and get the true isotype?

In [15]:
# Downselect only to V genes that are in our data
# We will never get matches on the rest. Clustering will always fail.
invalid_v_genes = set(df["v_gene"].unique()) - set(
    helpers.all_observed_v_genes()[GeneLocus.BCR]
)
logger.warning(f"Dropping CoV-AbDab V genes that aren't in our data: {invalid_v_genes}")
df = df[df["v_gene"].isin(helpers.all_observed_v_genes()[GeneLocus.BCR])]

2022-11-15 05:17:30,128 - covabdab_add_igblast_annotations.iypnb - WARNING - Dropping CoV-AbDab V genes that aren't in our data: set()


# Export

In [16]:
df

,CDRH3,j_gene,VHorVHH,Binds to,Doesn't Bind to,Neutralising Vs,Not Neutralising Vs,Protein + Epitope,Origin,cdr3_seq_aa_q_trim,cdr3_aa_sequence_trim_len,rownum,v_gene,cdr1_seq_aa_q_trim,cdr2_seq_aa_q_trim
0,ARERGYSGYGAAYYFDY,IGHJ4,EVQLLESGTEVKKPGSSVKVSCKASGGTFSSYAISWVRQAPGQGLE...,SARS-CoV2_WT;SARS-CoV2_Omicron-BA1,NaN,SARS-CoV2_WT;SARS-CoV2_Omicron-BA1,NaN,S; RBD,"Phage Display (Ab, human, immune - SARS-CoV2)",ARERGYSGYGAAYYFDY,17,0,IGHV1-69,GGTFSSYA,IIPILGIA
1,ASWLYGDPISFDY,IGHJ4,EVQLLESGPGLVKPSETLSLTCTVSGGSISNSSYYWGWIRQPPGKG...,SARS-CoV2_WT;SARS-CoV2_Omicron-BA1,NaN,SARS-CoV2_WT;SARS-CoV2_Omicron-BA1,NaN,S; RBD,"Phage Display (Ab, human, immune - SARS-CoV2)",ASWLYGDPISFDY,13,1,IGHV4-39,GGSISNSSYY,IYYSGST
2,AKSQGDYGGNSGAGYFQH,IGHJ1,EVQLVESGGGLVQPGRSLRLSCAASGFTFDEYAMHWVRQAPGKGLE...,SARS-CoV2_WT (weak);SARS-CoV2_Beta (weak);SARS...,NaN,SARS-CoV2_WT;SARS-CoV2_Gamma;SARS-CoV2_Omicron...,NaN,S; RBD,B-cells; SARS-CoV2_Gamma Human Patient,AKSQGDYGGNSGAGYFQH,18,2,IGHV3-9,GFTFDEYA,SSWNSGTI
3,ARLREFGDYYYYGMDV,IGHJ6,EVQLVQSGAEVKKPGESLKISCLGSGYSFTSYWIGWVRQMPGKGLE...,SARS-CoV2_WT (weak);SARS-CoV2_Beta,SARS-CoV2_WT,SARS-CoV2_Gamma;SARS-CoV2_Omicron-BA1 (weak),SARS-CoV2_WT,S; RBD,B-cells; SARS-CoV2_Gamma Human Patient,ARLREFGDYYYYGMDV,16,3,IGHV5-51,GYSFTSYW,IFPADSDT
4,ARDTTDDYYVI,IGHJ1,QVQLQESGPGLVKPSQTLSLTCTVSGGSISSGSYYWSWIRQPAGKG...,SARS-CoV2_WT;SARS-CoV2_Beta;SARS-CoV2_Gamma,NaN,SARS-CoV2_WT;SARS-CoV2_Gamma,SARS-CoV2_Omicron-BA1,S; RBD,B-cells; SARS-CoV2_Gamma Human Patient,ARDTTDDYYVI,11,4,IGHV4-61,GGSISSGSYY,IYTSGST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6839,ARGESGSPYGMDV,IGHJ6,EVQLLESGGGLIQPGGSLRLSCAASGLTVSSNYMSWVRQAPGKGLE...,SARS-CoV2_WT,NaN,SARS-CoV2_WT,NaN,S; RBD,"Phage Display (Ab, human, immune - CoV2_WT)",ARGESGSPYGMDV,13,6839,IGHV3-53,GLTVSSNY,IYSGGST
6840,AAPSCSRTICSDGFDI,IGHJ3,EVQLVQSGPEVKKPGTSVKVACKASGFTFITPVSMQWVRQARGQRL...,SARS_CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,SARS_CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,NaN,S; RBD,B-cells; SARS-CoV2 Human Vaccinees (BNT162b2),AAPSCSRTICSDGFDI,16,6840,IGHV1-58,GFTFITPVS,IVLASGNT
6841,AREGRRYGSGWYISTGYFDY,IGHJ4,QVQLVQSGAEVKKPGSSVRVSCKASGGTFSTYPISWVRQAPGQGLE...,SARS-CoV2_WT,NaN,SARS-CoV2_WT,NaN,S; RBD,B-cells; SARS-CoV2_WT Human Patient,AREGRRYGSGWYISTGYFDY,20,6841,IGHV1-69,GGTFSTYP,IIPIFGTA
6842,AIHGGTYYYDKNILA,IGHJ4,EVQLVESGGGVVQPGTSLRLSCAASGFSFSHYVMYWVRQAPGKGLD...,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,SARS-CoV2_Omicron,SARS-CoV2_WT;SARS-CoV2_Alpha;SARS-CoV2_Beta;SA...,SARS-CoV2_Omicron,S; RBD,B-cells (SARS-CoV2 Vaccinated Human),AIHGGTYYYDKNILA,15,6842,IGHV3-30,GFSFSHYV,ISFDGSSQ


In [17]:
df.columns

Index(['CDRH3', 'j_gene', 'VHorVHH', 'Binds to', 'Doesn't Bind to',
       'Neutralising Vs', 'Not Neutralising Vs', 'Protein + Epitope', 'Origin',
       'cdr3_seq_aa_q_trim', 'cdr3_aa_sequence_trim_len', 'rownum', 'v_gene',
       'cdr1_seq_aa_q_trim', 'cdr2_seq_aa_q_trim'],
      dtype='object')

In [18]:
df.drop(columns="rownum").to_csv(
    config.paths.base_data_dir / "CoV-AbDab_260722.filtered.annotated.tsv",
    sep="\t",
    index=None,
)